In [1]:
%matplotlib widget

from mpl_toolkits import mplot3d

import time
import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d
import matplotlib.cm as cm

### Plotting code, not relevant to solver ###
arrow_prop_dict = dict(mutation_scale=20, arrowstyle='->', shrinkA=0, shrinkB=0)
class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0, 0), (0, 0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        FancyArrowPatch.draw(self, renderer)

def draw_coordinate_frame(ax, T, R):
    tx, ty, tz = T
    new_x = R @ np.array([1, 0, 0.0]) + T
    new_y = R @ np.array([0, 1, 0.0]) + T
    new_z = R @ np.array([0, 0, 1.0]) + T
    
    a = Arrow3D([tx, new_x[0]], [ty, new_x[1]], [tz, new_x[2]], **arrow_prop_dict, color='r')
    ax.add_artist(a)
    a = Arrow3D([tx, new_y[0]], [ty, new_y[1]], [tz, new_y[2]], **arrow_prop_dict, color='b')
    ax.add_artist(a)
    a = Arrow3D([tx, new_z[0]], [ty, new_z[1]], [tz, new_z[2]], **arrow_prop_dict, color='g')
    ax.add_artist(a)
    
def draw_vec(ax, p1, p2, color):
    a = Arrow3D([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], **arrow_prop_dict, color=color)
    ax.add_artist(a)

In [2]:
def project_to_plane(origin, vec):
    ground_normal, ground_point = np.array([0, 0, 1]), np.array([0, 0, 0])
    t = (-ground_normal @ (origin - ground_point)) / (ground_normal @ vec)
    return t

In [3]:
data = np.load("all_data.npz", allow_pickle=True)
x_traj = data["x"]
target_trajs = data["target"]

data = np.load("../calibration/drone_target_sample.npz", allow_pickle=True)
pix_3d = data["pixel_drone"]
N = 5
dt = 0.1

In [13]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
colors = cm.rainbow(np.linspace(0, 1, N+1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
# from drone_mpc_forces_pro import DroneMPC as DroneMPCForces
# drone_mpc_forces = DroneMPCForces(N=N, dt=dt, load_solver=False)

In [9]:
from scipy.spatial.transform import Rotation

output_format =  'x{0:0%dd}' % (int(math.log(N) / math.log(10)) + 1)

i = 90  # 0 ... 120
x0 = x_traj[i]
T = x0[:3]

yaw = x0[-1]
R = Rotation.from_euler("XYZ", [0, 0, yaw])

target_traj = target_trajs[i][:N]  # N x 3 target positions over horizon of N

# X_mpc, U_mpc = drone_mpc_forces.solve(x0, pix_3d, target_traj)


# # Plot 
# target_x = target_traj[:, 0]
# target_y = target_traj[:, 1]
# target_z = target_traj[:, 2]
# ax.plot(target_x, target_y, target_z, label='target path')
# for i in range(N):
#     draw_coordinate_frame(ax, T=target_traj[i, :], R=np.eye(3))

# ax.plot(X_mpc[:,0], X_mpc[:,1], X_mpc[:,2], label='drone path')
# for i in range(X_mpc.shape[0]):
#     rot = Rotation.from_euler("ZYX", [X_mpc[i, -1], 0, 0]).as_matrix()
#     draw_coordinate_frame(ax, T=X_mpc[i, :3], R=rot)
#     desired_vec = rot @ pix_3d
#     t = project_to_plane(X_mpc[i, :3], desired_vec)
#     draw_vec(ax, X_mpc[i, :3], desired_vec*t + X_mpc[i, :3], color=colors[i])
# draw_coordinate_frame(ax, T=T, R=R.as_matrix())

# # vec from drone to target
# true_vec = target_traj[0] - x0[:3]
# draw_vec(ax, x0[:3], target_traj[0], color='r')

# # desired vec (center of image plane)
# yaw = x0[-1]
# vel = np.array([x0[3], x0[4], x0[5]])
# vel = vel / np.linalg.norm(vel)
# theta = math.asin(-vel[1])  # pitch
# # print([yaw, theta, phi])
# print(R.as_euler("ZYX"))
# fake_R = Rotation.from_euler('ZYX', [yaw, 0, 0])
# desired_vec = fake_R.as_matrix() @ pix_3d
# t = project_to_plane(T, desired_vec)
# draw_vec(ax, T, desired_vec*t + T, color='g')

# ax.legend()
# plt.show()
# print(X_mpc[:, :3])

In [10]:
x0.shape

(7,)

In [11]:
from drone_mpc import DroneMPC
con = DroneMPC(N=N)
start_time = time.time()

X_mpc, U_mpc = con.solve(
    x0=x0, u0=None, target_pixel=pix_3d,
    phi0=0, target_traj=target_traj, user_sq_dist=0.5)
print("Time elapsed: %.2f" % (time.time() - start_time) )

[[1.  0.  0.  0.1 0.  0.  0. ]
 [0.  1.  0.  0.  0.1 0.  0. ]
 [0.  0.  1.  0.  0.  0.1 0. ]
 [0.  0.  0.  1.  0.  0.  0. ]
 [0.  0.  0.  0.  1.  0.  0. ]
 [0.  0.  0.  0.  0.  1.  0. ]
 [0.  0.  0.  0.  0.  0.  1. ]]
[[0.005 0.    0.    0.   ]
 [0.    0.005 0.    0.   ]
 [0.    0.    0.005 0.   ]
 [0.1   0.    0.    0.   ]
 [0.    0.1   0.    0.   ]
 [0.    0.    0.1   0.   ]
 [0.    0.    0.    0.1  ]]
Time elapsed: 1.36


In [ ]:
X_mpc

In [14]:
target_x = target_traj[:, 0]
target_y = target_traj[:, 1]
target_z = target_traj[:, 2]
ax.plot(target_x, target_y, target_z, label='target path')
for i in range(N):
    draw_coordinate_frame(ax, T=target_traj[i, :], R=np.eye(3))
    
ax.plot(X_mpc[:,0], X_mpc[:,1], X_mpc[:,2], label='drone path')
for i in range(X_mpc.shape[0]):
    rot = Rotation.from_euler("ZYX", [X_mpc[i, -1], 0, 0]).as_matrix()
    draw_coordinate_frame(ax, T=X_mpc[i, :3], R=rot)
    desired_vec = rot @ pix_3d
    t = project_to_plane(X_mpc[i, :3], desired_vec)
    draw_vec(ax, X_mpc[i, :3], desired_vec*t + X_mpc[i, :3], color=colors[i])
draw_coordinate_frame(ax, T=T, R=R.as_matrix())

# vec from drone to target
true_vec = target_traj[0] - x0[:3]
draw_vec(ax, x0[:3], target_traj[0], color='r')

# desired vec (center of image plane)
yaw = x0[-1]
vel = np.array([x0[3], x0[4], x0[5]])
vel = vel / np.linalg.norm(vel)
theta = math.asin(-vel[1])  # pitch
# print([yaw, theta, phi])
print(R.as_euler("ZYX"))
fake_R = Rotation.from_euler('ZYX', [yaw, 0, 0])
desired_vec = fake_R.as_matrix() @ pix_3d
t = project_to_plane(T, desired_vec)
draw_vec(ax, T, desired_vec*t + T, color='g')

ax.legend()
plt.show()

[0.60942367 0.         0.        ]
